# Traffic Prediction Model

This notebook contains the ML model for predicting traffic patterns.


In [ ]:
# TODO: Implement traffic prediction model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib
